<a href="https://colab.research.google.com/github/aetev/Learning-stuff-/blob/main/WGAN-GP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow_addons
#!pip install pydub
!pip install librosa


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_addons as tfa
import os
from IPython.display import Audio

2023-08-08 11:21:36.922321: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [4]:
import librosa
import numpy as np

def downsample_array(arr, factor, axis):
    return arr.take(np.arange(0, arr.shape[axis], factor), axis)

def create_sliding_window(array, window_size, stride):
    num_windows = (len(array) - window_size) // stride + 1
    sliding_windows = np.lib.stride_tricks.sliding_window_view(array, (window_size,))

    return sliding_windows[::stride]

In [5]:
reduction = 12
wav_file = '/tmp/bass samples/NBKoanbandstuff.wav'
audio, sr = librosa.load(wav_file, sr=None)
audio_dev = np.std(audio)
audio = audio/audio_dev
result_array = create_sliding_window(audio,44100,100)
result_array = np.expand_dims(result_array, axis=2)
result_array = downsample_array(result_array,reduction,1)

In [6]:


audio_data = result_array[100].ravel()
# Play the audio within the Jupyter Notebook
Audio(data=audio_data, rate=sr/reduction)

In [7]:
#x_train = noise
y_train = result_array
print(y_train[0].shape)

(3675, 1)


In [8]:
class ResNetBlock(layers.Layer):
    def __init__(self, filters,kernel_size=3, strides=1,dilation_rate=1):
        super(ResNetBlock, self).__init__()
        self.conv1 = layers.Conv1D(filters, kernel_size, strides=strides,dilation_rate=dilation_rate, padding='same')
        self.conv2 = layers.Conv1D(filters, kernel_size, padding='same')

        if strides != 1:
            self.residual = layers.Conv1D(filters, 1, strides=strides)
        else:
            self.residual = lambda x: x

    def call(self, inputs, training=False):
        
        x = self.conv1(inputs)
        x = tf.nn.relu(x)
        x = self.conv2(x)

        r = self.residual(inputs)

        x += r
        
        return tf.nn.relu(x)

In [9]:
class ResNetBlockup(layers.Layer):
    def __init__(self, filters, kernel_size=3, strides=1, dilation_rate=1):
        super(ResNetBlockup, self).__init__()
        self.conv1 = layers.Conv1DTranspose(filters, kernel_size, strides=strides, dilation_rate=dilation_rate, padding='same')
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv1DTranspose(filters, kernel_size, padding='same')
        self.bn2 = layers.BatchNormalization()

        if strides != 1:
            self.residual = layers.Conv1DTranspose(filters, 1, strides=strides)
        else:
            self.residual = lambda x: x

    def call(self, inputs, training=False):
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv2(x)
        x = self.bn2(x, training=training)

        r = self.residual(inputs)

        x += r
        return tf.nn.relu(x)

In [10]:
def build_discriminator():
    input_series = layers.Input(shape=(None,1))


    x = layers.Conv1D(128, 4, strides =1, padding='same')(input_series)
    
    x = layers.Conv1D(128, 4, strides =2, padding='same')(x)
    
    x = layers.Conv1D(128, 4, strides =4, padding='same')(x)
    
    x = layers.Conv1D(128, 4, strides =8, padding='same')(x)
    
    x = layers.Conv1D(128, 4, strides =12, padding='same')(x)
    
    x = layers.Conv1D(128, 4, strides =24, padding='same')(x)
    
    x = layers.Conv1D(128, 4, strides =48, padding='same')(x)
    
    x = layers.Conv1D(512, 4, strides =96, padding='same')(x)






    # Global pooling
    pooled_output = layers.GlobalAveragePooling1D()(x)

    # Dense layer
    dense_output = layers.Dense(64, activation='relu')(pooled_output)

    # Dense layer
    dense_output = layers.Dense(1, activation='linear')(dense_output)

    model = tf.keras.models.Model(inputs=input_series, outputs=dense_output)
    return model

discriminator = build_discriminator()
discriminator.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, None, 128)         640       
                                                                 
 conv1d_1 (Conv1D)           (None, None, 128)         65664     
                                                                 
 conv1d_2 (Conv1D)           (None, None, 128)         65664     
                                                                 
 conv1d_3 (Conv1D)           (None, None, 128)         65664     
                                                                 
 conv1d_4 (Conv1D)           (None, None, 128)         65664     
                                                                 
 conv1d_5 (Conv1D)           (None, None, 128)         65664 

2023-08-08 11:21:39.461796: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-08 11:21:39.465486: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-08 11:21:39.465665: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [11]:
def build_generator():
    input_series = layers.Input(shape=(None,1))

    x = ResNetBlock(128,4,strides=1,dilation_rate=24)(input_series)
    
    x = layers.Dropout(.2)(x)

    x = ResNetBlock(128,4,strides=1,dilation_rate=12)(x)
    
    x = layers.Dropout(.2)(x)

    x = ResNetBlock(128,4,strides=1,dilation_rate=6)(x)
    
    x = layers.Dropout(.2)(x)

    x = ResNetBlock(128,4,strides=1,dilation_rate=4)(x)
    
    x = layers.Dropout(.2)(x)

    x = ResNetBlock(128,4,strides=1,dilation_rate=2)(x)
    
    x = layers.Dropout(.2)(x)

    x = ResNetBlock(128,4,strides=1,dilation_rate=1)(x)

    x = layers.Conv1D(1,1)(x)



    model = tf.keras.models.Model(inputs=input_series, outputs=x)
    return model

generator = build_generator()
generator.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None, 1)]         0         
                                                                 
 res_net_block (ResNetBlock)  (None, None, 128)        66304     
                                                                 
 dropout (Dropout)           (None, None, 128)         0         
                                                                 
 res_net_block_1 (ResNetBloc  (None, None, 128)        131328    
 k)                                                              
                                                                 
 dropout_1 (Dropout)         (None, None, 128)         0         
                                                                 
 res_net_block_2 (ResNetBloc  (None, None, 128)        131328    
 k)                                                        

In [12]:
# Compile models
#generator_optimizer = tf.keras.optimizers.Adam(0.00004)
#discriminator_optimizer = tf.keras.optimizers.Adam(0.00004)

# Set your desired hyperparameters
learnRateD = 0.0002
learnRateG = 0.001
decayRateD = 0.9
decayRateG = 0.9
epsilonD = 1e-10
epsilonG = 1e-10

# Create the optimizers with the set hyperparameters
generator_optimizer = tf.keras.optimizers.RMSprop(learning_rate=learnRateG, rho=decayRateG, epsilon=epsilonG, centered=False)
discriminator_optimizer = tf.keras.optimizers.RMSprop(learning_rate=learnRateD, rho=decayRateD, epsilon=epsilonD, centered=False)
#generator_optimizer = tf.keras.optimizers.experimental.SGD(1e-4)
#discriminator_optimizer = tf.keras.optimizers.experimental.SGD(1e-4)

In [13]:
def gradient_penalty(real, fake, discriminator):
    batch_size = real.shape[0]
    epsilon = tf.random.uniform([batch_size, 1, 1], 0.0, 1.0)
    interpolated = epsilon * real + (1 - epsilon) * fake

    with tf.GradientTape() as tape:
        tape.watch(interpolated)
        pred = discriminator(interpolated, training=True)

    gradients = tape.gradient(pred, [interpolated])[0]
    norm = tf.sqrt(tf.reduce_sum(tf.square(gradients), axis=[1, 2]))
    gp = tf.reduce_mean((norm - 1.0) ** 2)

    return gp

In [14]:
def discriminator_loss(real_output, fake_output, gradient_penalty):
    return tf.reduce_mean(fake_output) - tf.reduce_mean(real_output) + gradient_penalty

def generator_loss(fake_output):
    return -tf.reduce_mean(fake_output)



In [15]:
def clip_discriminator_weights(discriminator):
    for l in discriminator.layers:
        weights = l.get_weights()
        weights = [tf.clip_by_value(w, -0.01, 0.01) for w in weights]
        l.set_weights(weights)

In [16]:
def print_img(generator_model):
    # Generate and save sample images
    noise = tf.random.normal([10, 100])
    sampled_labels = tf.constant([[i % 10] for i in range(10)], dtype=tf.int32)
    generated_images = generator_model.predict([noise, sampled_labels])
    fig, axs = plt.subplots(1, 10, figsize=(10, 10))
    for i in range(10):
        axs[i].imshow(generated_images[i], cmap="gray")
        axs[i].axis("off")
    plt.show()

In [17]:
noise = tf.random.normal(shape=(1,10000,1))

test = generator.predict(noise)
print(test.shape)

2023-08-08 11:21:41.111911: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600


1/1 [==============================] - 1s 1s/step
(1, 10000, 1)


In [ ]:

#@tf.function
def train_step(target_audios):


  for i in range(5):

      # Get the shape of the target_audios tensor
      shape = tf.shape(target_audios)


      # Generate noise using tf.random.normal()
      noise = tf.random.normal((shape))
      generated_audio = generator(noise, training=True)
      with tf.GradientTape() as disc_tape:

          real_output = discriminator(target_audios, training=True)
          fake_output = discriminator(generated_audio, training=True)
          gp = gradient_penalty(target_audios, generated_audio, discriminator)
          disc_loss = discriminator_loss(real_output, fake_output, gp)

      gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
      discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
      #clip_discriminator_weights(discriminator)

      if i ==0:
          weights = discriminator.get_weights()


  with tf.GradientTape() as gen_tape:
    noise = tf.random.normal(shape=(target_audios.shape))
    generated_audio = generator(noise, training=True)
    fake_output = discriminator(generated_audio, training=True)
    gen_loss = generator_loss(fake_output)

  gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
  generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))

  discriminator.set_weights(weights)


  tf.print("disc_loss",disc_loss,'gen_loss',gen_loss)


def train(generator, discriminator
          , epochs, batch_size):
    for epoch in range(epochs):
        for batch in range(len(y_train) // batch_size):
            #images = x_train[batch * batch_size: (batch+1) * batch_size]
            target_audios = y_train[batch * batch_size: (batch+1) * batch_size]

            train_step(target_audios)

        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1}/{epochs}")
            test = generator.predict(noise)
            audio_data = test.ravel()
            # Play the audio within the Jupyter Notebook
            Audio(data=audio_data, rate=sr/reduction)


# Train the GAN
EPOCHS = 2000000
BATCH_SIZE = 10
num_unrolling_steps = 20  # Set the desired number of unrolling steps
train(generator, discriminator, EPOCHS, BATCH_SIZE)

2023-08-08 11:21:42.231582: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x4299cd20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-08 11:21:42.231610: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 SUPER, Compute Capability 7.5
2023-08-08 11:21:42.234803: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-08 11:21:42.323995: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


disc_loss -5.87054491 gen_loss 19.6036568
disc_loss -874.223511 gen_loss 2509.10376
disc_loss 0.330419242 gen_loss 0.652748704
disc_loss 0.281469882 gen_loss 0.726069689
disc_loss 0.226503074 gen_loss 0.764575303
disc_loss 0.165609121 gen_loss 0.850236058
disc_loss 0.100284338 gen_loss 0.91120851
disc_loss 0.0121122599 gen_loss 1.01947248
disc_loss -0.129227281 gen_loss 1.12867236
disc_loss -0.252874255 gen_loss 1.25067163
disc_loss -0.419613421 gen_loss 1.4767741
disc_loss -0.640644431 gen_loss 1.86514699
disc_loss -1.13830495 gen_loss 2.40947986
disc_loss -1.52232993 gen_loss 3.04806471
disc_loss -2.54595804 gen_loss 4.45854855
disc_loss -4.23937702 gen_loss 7.65778828
disc_loss -6.21899176 gen_loss 11.3234959
disc_loss -5.7644062 gen_loss 10.8335505
disc_loss -2.79889441 gen_loss 5.33838415
disc_loss -0.155564427 gen_loss 0.977668405
disc_loss 0.74966538 gen_loss 0.354519337
disc_loss 0.527735054 gen_loss 0.316443771
disc_loss -0.941367686 gen_loss 3.02097678
disc_loss -2.69819927 g

In [ ]:
test = generator.predict(noise)
audio_data = test.ravel()
# Play the audio within the Jupyter Notebook
Audio(data=audio_data, rate=sr/reduction)